In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
mnist = tf.keras.datasets.mnist

In [18]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [19]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [20]:
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [21]:
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [52]:
class UniModel(Model):
    def __init__(self):
        super(UniModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

In [53]:
model = UniModel()

In [55]:
model.stateful

False

In [56]:
loss_func = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

In [57]:
train_loss = tf.keras.metrics.Mean(name = 'train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
test_loss = tf.keras.metrics.Mean(name = 'test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [58]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_func(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss(loss)
    train_accuracy(labels, predictions)

In [59]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_func(labels, predictions)
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [60]:
EPOCHES = 5
for epoch in range(EPOCHES):
    # 在下一个epoch开始时，重置评估指标
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    for images, labels in train_ds:
        train_step(images, labels)
    for images, labels in test_ds:
        test_step(images, labels)
    template = "epoch {}, loss {}, accuracy {}，test_loss {}, test_accuracy {}"
    print(template.format(epoch+1, train_loss.result(), train_accuracy.result()*100, 
                          test_loss.result(), test_accuracy.result() * 100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

epoch 1, loss 0.1310039460659027, accuracy 96.12667083740234，test_loss 0.05859571322798729, test_accuracy 98.04999542236328
epoch 2, loss 0.04092904552817345, accuracy 98.72166442871094，test_loss 0.04863058403134346, test_accuracy 98.38999938964844
epoch 3, loss 0.02052241750061512, accuracy 99.36500549316406，test_loss 0.058564286679029465, test_accuracy 98.18000030517578
epoch 4, loss 0.012330768629908562, accuracy 99.58999633789062，test_loss 0.05908020958304405, test_accuracy 98.33999633789062
epoch 5